In [0]:
customers_df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/customers")
display(customers_df)


In [0]:
orders_df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/orders")
orders_df = orders_df.filter(orders_df['order_status']=="COMPLETE")
display(orders_df)

In [0]:
stores_df = spark.read.format("delta").load("dbfs:/user/hive/warehouse/stores")
display(stores_df)

In [0]:
df_joined = orders_df.join(
    stores_df,
    orders_df['store_id'] == stores_df['store_id'],
    how="left"
)

selected_columns = df_joined['order_id','order_datetime','customer_id' ,'store_name']
selected_columns.display()

# Define the output path for the Parquet file
output_path = "dbfs:/user/hive/warehouse/silver-order"

# Write the DataFrame to Parquet format
selected_columns.write.mode('overwrite').parquet(output_path)

Databricks data profile. Run in Databricks to view.

In [0]:
silver_order_df = spark.read.format("parquet").load("dbfs:/user/hive/warehouse/silver-order")
display(silver_order_df)

In [0]:
spark.sql("DROP TABLE IF EXISTS orders_tv")
silver_order_df.createTempView("orders_tv")

In [0]:
%sql
select distinct(store_name) from orders_tv

In [0]:
%sql
select current_database()


In [0]:
spark.sql("SHOW TABLES IN default")
